In [1]:
!rm -rf models/checkpoints

In [2]:
from aimoppo import AIMOPPOTrainer
from datasets import load_dataset, Dataset
import re

[2024-06-01 15:04:40,686] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.1.0), only 1.0.0 is known to be compatible


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [3]:
dataset = load_dataset("json", data_dir="data/MATH")
def is_real_number(text):
    try:
        float(text)
        return True
    except Exception:
        return False
def extract_answer(text):
    try:
        match = re.search(r"\\boxed{(.+?)}", text)
        return match.group(1)
    except Exception:
        return None

dataset_with_answer = dataset.map(lambda x: {"problem": x["problem"], "answer": extract_answer(x["solution"])})
dataset_with_answer = dataset_with_answer.filter(lambda x: is_real_number(x["answer"]))
dataset_with_answer = dataset_with_answer.rename_column("problem", "query")

Resolving data files:   0%|          | 0/7500 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/5000 [00:00<?, ?it/s]

In [4]:
train_dataset = dataset_with_answer['train']
val_dataset = dataset_with_answer['test']

test_dataset = Dataset.from_csv("data/val.csv")
test_dataset = test_dataset.rename_column("problem", "query")
test_dataset = test_dataset.remove_columns(["id"])

train_dataset = train_dataset.shuffle()
test_dataset = test_dataset.shuffle()

train_dataset = train_dataset.select(range(1000))
val_dataset = val_dataset.select(range(20))


trainer = AIMOPPOTrainer(
    "configs/costar_cot_1shot.yaml",
    train_dataset,
    val_dataset,
    test_dataset,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chengpong1127. Use `wandb login --relogin` to force relogin


In [8]:
trainer.train()

  0%|          | 0/310 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 8.46 GiB. GPU 0 has a total capacity of 23.69 GiB of which 7.15 GiB is free. Process 447706 has 16.49 GiB memory in use. Of the allocated memory 13.03 GiB is allocated by PyTorch, and 3.15 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF